In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.cluster import KMeans

In [ ]:
data = pd.read_excel('Online Retail.xlsx')
data.head()

In [ ]:
data.info()

In [ ]:
data = data[data['CustomerID'].notnull()]

In [ ]:
data.info()

In [ ]:
data['InvoiceDay'] = data['InvoiceDate'].apply(lambda x: dt.datetime(x.year, x.month, x.day))

In [ ]:
data.head()

In [ ]:
dt.timedelta(1)

In [ ]:
data['TotalSum'] = data['Quantity'] * data['UnitPrice']
data.head()

In [ ]:
pin_date = dt.date.today()
pin_date = pd.Timestamp(pin_date) 

In [ ]:
rfm = data.groupby('CustomerID').agg({
    'InvoiceDay': lambda x: (pin_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'
})
rfm

In [ ]:
data[data['CustomerID'] == 12346.0]

In [ ]:
rfm.rename(columns= {
    'InvoiceDay': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalSum': 'Monetary'
}, inplace=True)
rfm

In [ ]:
r_labels = range(4, 0, -1) #[4, 3, 2, 1]
r_groups = pd.qcut(rfm['Recency'], q=4, labels=r_labels)
f_labels = range(1, 5) # [1, 2, 3, 4]
f_groups = pd.qcut(rfm['Frequency'], q=4, labels=f_labels)
m_labels = range(1, 5)
m_groups = pd.qcut(rfm['Monetary'], q=4, labels=m_labels)

In [ ]:
rfm['R'] = r_groups.values
rfm['F'] = f_groups.values
rfm['M'] = m_groups.values
rfm

In [ ]:
X = rfm[['R', 'F', 'M']]
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300)
kmeans.fit(X)

In [ ]:
kmeans.labels_

In [ ]:
rfm['kmeans_cluster'] = kmeans.labels_

In [ ]:
rfm

In [ ]:
rfm[rfm['kmeans_cluster'] == 4]

In [ ]:
customers_g4 = list(rfm[rfm['kmeans_cluster'] == 4].index)
top_products_g4 = data.query('CustomerID == @customers_g4').groupby(['StockCode'])['InvoiceNo'].count().sort_values(ascending=False).head(10)
top_products_g4

In [ ]:
customers_g4 = list(rfm[rfm['kmeans_cluster'] == 4].index)
top_products_g4 = data.query('CustomerID == @customers_g4').groupby(['StockCode'])['InvoiceNo'].count().sort_values(ascending=False).head(10)
top_products_g4

In [ ]:
rfm[rfm['kmeans_cluster'] == 4].mean()

In [ ]:
rfm.mean()